In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch
bnb= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [2]:
model=AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=bnb,
    trust_remote_code=True
).to('cuda')
model.config.use_sliding_window = False
model = torch.compile(model)
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import fitz

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page_num, page in enumerate(doc, start=1):
            page_text = page.get_text()
            text += f"\n\n--- Page {page_num} ---\n\n{page_text}"
    return text

# Example usage
pdf_file_path = "your_file.pdf"
pdf_text = extract_text_from_pdf(pdf_file_path)


In [23]:
path="/home/pa2/ml/tf_gpu/OCR/extracted_data/B16. Trademark Assignment Agreement.txt"
text=''
with open(path,'r') as file:
    text=file.read()

In [24]:
text_chunks=text.split("Page")
print(len(text_chunks))

12


In [25]:
chunk_summaries=""""""
cnt=0
for i in text_chunks:
    cnt+=1
    print("Page",cnt)
    chat=[
        {
            "role":"user",
            "content":f"""
                    Read the following passage and distill its core ideas into a clear, 
                    conversational summary of no more than two sentences, 
                    as if you were explaining it to a colleague in formal manner.
    
                    <text>
                    {i}
                    </text>
                    Summary:
            """
        }
    ]
    message_prompt=tokenizer.apply_chat_template(chat,tokenize=False)
    prompt=tokenizer([message_prompt],return_tensors='pt').to('cuda')
    out=model.generate(input_ids=prompt['input_ids'],attention_mask=prompt['attention_mask'],max_new_tokens=512).squeeze()
    chunk_summaries+=tokenizer.decode(out,skip_special_tokens=True)
    chunk_summaries+='\n\n'
print(chunk_summaries)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

user

Read the following passage and distill its core ideas into a clear, 
                    conversational summary of no more than two sentences, 
                    as if you were explaining it to a colleague in formal manner.
    
                    <text>
                    
                    </text>
                    Summary:assistant

I'm happy to help, but there is no passage to summarize. Please provide the text, and I'll be happy to distill its core ideas into a clear and concise summary for you.

system

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

user

Read the following passage and distill its core ideas into a clear, 
                    conversational summary of no more than two sentences, 
                    as if you were explaining it to a colleague in formal manner.
    
                    <text>
                     1:
HETRYTON STIRITS
Government of Maharashtra
0.
exr

In [26]:
import re

def extract_assistant_to_system(text):
    # Regex pattern to capture text between 'assistant' and 'system' (non-greedy, multiline)
    pattern = r"assistant\s+(.*?)\s+system"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    return matches

# Example usage
input_text = chunk_summaries

results = extract_assistant_to_system(input_text)

# Print each extracted block
chunk_summaries=""""""
for i, result in enumerate(results, 1):
    print(f"--- Summary Block {i} ---\n{result.strip()}\n")
    chunk_summaries+=result.strip()
    chunk_summaries+='\n\n'


--- Summary Block 1 ---
I'm happy to help, but there is no passage to summarize. Please provide the text, and I'll be happy to distill its core ideas into a clear and concise summary for you.

--- Summary Block 2 ---
Here's a two-sentence summary of the passage in a formal, conversational tone:

I'd like to bring to your attention a document related to a trademark assignment, specifically an E-secured bank receipt (E-SBTR) for a transaction between Protean eGov Technologies Limited and National Securities Depository Limited, dated 15th October 2022, with a value of Rs. 10,00,000. This document is a stamp duty receipt for the assignment of a trademark, which is valid for six months from the date of payment.

--- Summary Block 3 ---
Here is a two-sentence summary of the passage in a conversational tone:

I'd like to brief you on the Trademarks Assignment Agreement between Protean EGov Technologies Limited and National Securities Department. The agreement, effective as of December 12, 202

In [22]:
def final_summary(text):
    summary=""""""
    message=[
        {
            'role':'user',
            "content":f"""
            You are a legal summarization assistant. Your task is to review multiple section-wise summaries of a legal document and generate a single, clear, and factually accurate final summary.

            ✦ The final summary must be:
            • Coherent and logically structured
            • Free of repetition or redundancy
            • Written in a formal and objective tone
            • Faithful to the original content—do not infer or exaggerate any conclusions
            
            ✦ Accurately reflect:
            • The nature of the dispute or document
            • The positions of the parties (if applicable)
            • The key legal or factual findings
            • The final outcome, ruling, or resolution
            
            Avoid subjective interpretation. Only include information that is explicitly present in the provided section summaries.
            <section_summary>
            {text}
            </section_summary>
            summary:
            
            """
        }
    ]
    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    inp=tokenizer([prompt],return_tensors='pt').to('cuda')
    out=model.generate(input_ids=inp['input_ids'],attention_mask=inp['attention_mask'],max_new_tokens=2048).squeeze()
    summary=tokenizer.decode(out,skip_special_tokens=True)
    return summary
print(final_summary(chunk_summaries))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

user

You are a legal summarization assistant. Your task is to review multiple section-wise summaries of a legal document and generate a single, clear, and factually accurate final summary.

            ✦ The final summary must be:
            • Coherent and logically structured
            • Free of repetition or redundancy
            • Written in a formal and objective tone
            • Faithful to the original content—do not infer or exaggerate any conclusions
            
            ✦ Accurately reflect:
            • The nature of the dispute or document
            • The positions of the parties (if applicable)
            • The key legal or factual findings
            • The final outcome, ruling, or resolution
            
            Avoid subjective interpretation. Only include information that is explicitly present in the provided section summaries.
            <section_summary>
            I'm happy t

In [27]:
def final_summary(text):
    summary=""""""
    message=[
        {
            'role':'user',
            "content":f"""
            You are a legal summarization assistant. Your task is to review multiple section-wise summaries of a legal document and generate a single, clear, and factually accurate final summary.

            ✦ The final summary must be:
            • Coherent and logically structured
            • Free of repetition or redundancy
            • Written in a formal and objective tone
            • Faithful to the original content—do not infer or exaggerate any conclusions
            
            ✦ Accurately reflect:
            • The nature of the dispute or document
            • The positions of the parties (if applicable)
            • The key legal or factual findings
            • The final outcome, ruling, or resolution
            
            Avoid subjective interpretation. Only include information that is explicitly present in the provided section summaries.
            <section_summary>
            {text}
            </section_summary>
            summary:
            
            """
        }
    ]
    prompt=tokenizer.apply_chat_template(message,tokenize=False)
    inp=tokenizer([prompt],return_tensors='pt').to('cuda')
    out=model.generate(input_ids=inp['input_ids'],attention_mask=inp['attention_mask'],max_new_tokens=2048).squeeze()
    summary=tokenizer.decode(out,skip_special_tokens=True)
    return summary
print(final_summary(chunk_summaries))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 09 May 2025

user

You are a legal summarization assistant. Your task is to review multiple section-wise summaries of a legal document and generate a single, clear, and factually accurate final summary.

            ✦ The final summary must be:
            • Coherent and logically structured
            • Free of repetition or redundancy
            • Written in a formal and objective tone
            • Faithful to the original content—do not infer or exaggerate any conclusions
            
            ✦ Accurately reflect:
            • The nature of the dispute or document
            • The positions of the parties (if applicable)
            • The key legal or factual findings
            • The final outcome, ruling, or resolution
            
            Avoid subjective interpretation. Only include information that is explicitly present in the provided section summaries.
            <section_summary>
            I'm happy t